Class: DSC 670<br>
Name: Eros Perez<br>
Week: 9<br>
Assignment: Build your first model, fully fine tune and demonstrate metrics from chapter 7

## My WFM Custom Model

Use Case<br>
A fine-tuned LLM predicts weekly contact center demand for 2025 based on a synthetic time series (2020–2024) that includes:<br>
Linear trend (e.g., business growth)<br>
Quarterly seasonality<br>
Cyclical outliers in January and August (marketing pushes)<br>

Accept prompts like:<br>
“Forecast demand for 2025 and explain variances"
Returns: forecast, actuals, MAPE, MAE, weekly variance, and average total variance.<br>
Handles zero-code interface via language prompt.<br>
Uses RAG to augment responses with workforce forecasting knowledge.

### Fine Tune Model

In this part of the code I fine tune my custom model. I had to go through several iterations of the examples so that my output actually displays forecasts, actuals and an analysis of the data set.

In [103]:
import os
import json
from openai import OpenAI

# 1: Initialize client 
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 2: Prepare training data
examples = [
    # TREND ANALYSIS
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in analyzing time series data."},
            {"role": "user", "content": "Here is weekly contact volume: [1000, 1020, 1040, 1070, 1100, 1140, 1180]. What trend do you observe?"},
            {"role": "assistant", "content": "There is a steady upward trend in volume, increasing by about 30–40 contacts per week."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in analyzing time series data."},
            {"role": "user", "content": "Volume data: [3000, 2950, 2920, 2900, 2850, 2800]. What is the trend here?"},
            {"role": "assistant", "content": "There is a clear downward trend, decreasing by about 50–100 contacts per week."}
        ]
    },
    # SEASONALITY
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in analyzing time series data."},
            {"role": "user", "content": "Monthly volume: [3200, 3300, 3500, 3700, 3900, 4100, 3900, 3700, 3400, 3100, 2900, 2700]. Is there seasonality?"},
            {"role": "assistant", "content": "Yes, there is a seasonal pattern with peaks in summer and declines in winter, suggesting Q2–Q3 seasonality."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in analyzing time series data."},
            {"role": "user", "content": "Quarterly volume: [8000, 12000, 9500, 7000, 8500, 12500, 9700, 7200]. Any seasonality?"},
            {"role": "assistant", "content": "Yes, the second quarter shows a consistent spike across both years, indicating quarterly seasonality."}
        ]
    },
    # OUTLIERS
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in analyzing time series data."},
            {"role": "user", "content": "Data: [1200, 1220, 1240, 3000, 1260, 1280]. Is there an outlier?"},
            {"role": "assistant", "content": "Yes, 3000 is an outlier. It significantly deviates from the other values, which are around 1200–1300."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in analyzing time series data."},
            {"role": "user", "content": "Weekly volume: [1100, 1120, 1140, 1160, 4000, 1180, 1200]. What's unusual here?"},
            {"role": "assistant", "content": "Week 5 shows an extreme spike to 4000. It is likely an outlier due to an unusual event."}
        ]
    },
    # COMBINED ANALYSIS
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst that specializes in trend and outlier detection."},
            {"role": "user", "content": "Data: [1500, 1520, 1550, 1580, 4000, 1610, 1640, 1670]. What do you observe?"},
            {"role": "assistant", "content": "There is an upward trend with a single large outlier at 4000."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You are a WFM data analyst."},
            {"role": "user", "content": "Monthly volume: [3200, 3300, 3500, 3600, 3000, 2900, 2800, 2700, 2600, 2500, 3300, 3400]. Trend?"},
            {"role": "assistant", "content": "There is a seasonal dip mid-year followed by a rebound in Q4, with a general pattern of decline and recovery."}
        ]
    },
    # CONTEXTUAL ANALYSIS
    {
        "messages": [
            {"role": "system", "content": "You analyze WFM time series patterns."},
            {"role": "user", "content": "Call volume jumped 70% in week 32. Could this be an outlier?"},
            {"role": "assistant", "content": "Yes, a 70% jump in a single week is likely an outlier, possibly driven by a marketing event or outage."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You analyze WFM time series patterns."},
            {"role": "user", "content": "Volume in January tends to spike after holidays. Last 3 years show +20%, +18%, +22%. What is this?"},
            {"role": "assistant", "content": "This is seasonal behavior—January shows a post-holiday spike of about 20% consistently."}
        ]
    },
    # PATTERN RECOGNITION
    {
        "messages": [
            {"role": "system", "content": "You detect contact center patterns."},
            {"role": "user", "content": "Here is the data: [2200, 2100, 2000, 1900, 2300, 2400, 2500]. What changed?"},
            {"role": "assistant", "content": "There was a declining trend followed by a reversal in week 5. Volume started increasing again."}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": "You detect contact center patterns."},
            {"role": "user", "content": "Volume history: [2500, 2550, 2500, 2600, 2550, 2500, 2450]. Is this stable or trending?"},
            {"role": "assistant", "content": "This pattern appears stable with small fluctuations, no clear trend."}
        ]
    },
    ### Output Format  --Added after running the first three examples
    {
  "messages": [
    {
      "role": "system",
      "content": "You are a workforce management forecasting assistant trained to analyze volume data."
    },
    {
      "role": "user",
      "content": "Here is weekly data:\n1000, 1020, 1045, 1070, 1100, 1080, 1120, 2500, 1140, 1160\nPlease forecast the next 10 weeks, generate realistic actuals, and display the results in table format. Then calculate MAPE, MAE, Weekly Variance, and Total Average Variance. Finally, summarize the trend, seasonality, and any outliers you observe."
    },
    {
      "role": "assistant",
      "content": "Week Date | Forecast | Actuals\n2025-01-06 | 1170 | 1150\n2025-01-13 | 1190 | 1185\n2025-01-20 | 1210 | 1230\n2025-01-27 | 1240 | 1225\n2025-02-03 | 1260 | 1240\n2025-02-10 | 1280 | 1295\n2025-02-17 | 1300 | 1325\n2025-02-24 | 1325 | 1280\n2025-03-03 | 1340 | 1360\n2025-03-10 | 1365 | 1380\n\nMAPE: 2.83%\nMAE: 31.0\nWeekly Variance: 28.6\nTotal Average Variance: 29.4\n\nSummary:\nThe forecast shows a steady upward trend in volume. There is slight weekly fluctuation but no major outliers in the forecasted period. The actuals align closely with the forecast, indicating strong model accuracy. A slight spike in mid-February may be cyclical. January typically shows strong volume after holidays, consistent with seasonal expectations."
    }
  ]
}
]

# 3: Save training examples locally
training_file_path = "wfm_ft_train.jsonl"
with open(training_file_path, "w") as f:
    for ex in examples:
        f.write(json.dumps(ex) + "\n")

# 4: Upload file using new API
uploaded_file = client.files.create(
    file=open(training_file_path, "rb"),
    purpose="fine-tune"
)
print("Training file ID:", uploaded_file.id)

# 5: Fine-tune model
fine_tune = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    model="gpt-3.5-turbo-1106",
    hyperparameters={"n_epochs": 3},
    suffix="wfm-predictor"
)
print("Fine-tuning job ID:", fine_tune.id)

Training file ID: file-58YzGU8ifZe4Cd9QmF924G
Fine-tuning job ID: ftjob-aZTzudhAuv2CpXEsm0CrySum


### Check which models are running

In [105]:
# Monitor status until job is complete
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')  # Replace with your OpenAI key env variable
)

# List all the FT jobs
ft_jobs = client.fine_tuning.jobs.list()

for ft_job in ft_jobs:
    print(ft_job.id, ft_job.status)

ftjob-aZTzudhAuv2CpXEsm0CrySum validating_files
ftjob-Ni37AUiSCprYvwSqYy4DwKGZ succeeded
ftjob-An9GvbgL4VqVjIUvYPKKVK8x succeeded
ftjob-t4dN1I5fNSFzfimv7Cc9NHZ2 failed
ftjob-f9i921QrGvnlCwCS6EtrRBa3 failed
ftjob-GcImBoZOr4QYEj4HNttmm73X failed
ftjob-l1EzzOuLEZWckU0phwmoU14S succeeded
ftjob-BxSqHkRBGhmX44mLU7UfzZ50 succeeded
ftjob-Zlc1wusynQnvqbtrWLLp5lvE succeeded
ftjob-UZpBGFkj2wVB1QGlB5ymbIk7 succeeded


### List of events of Fine Tune Job

In [108]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')  # Replace with your OpenAI key env variable
)

#ID's
id_1 = "ftjob-aZTzudhAuv2CpXEsm0CrySum"

ft_job_events = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=id_1, 
    limit=10)

# Loop through the events and print the details
for ft_job_event in ft_job_events:
    print(ft_job_event.id, ft_job_event.message)

ftevent-IViuXUTYc68bbJZZt31QcVhV Validating training file: file-58YzGU8ifZe4Cd9QmF924G
ftevent-XQhC6b0WCVTldbgr1eZmJM9d Created fine-tuning job: ftjob-aZTzudhAuv2CpXEsm0CrySum


### Monitor Speficic Model Progress Until Completed

In [110]:
# Monitor status until job is complete
track_fine_tuning(client, job_id="ftjob-aZTzudhAuv2CpXEsm0CrySum")

Status: succeeded
Elapsed Time: 4m 4s

✅ Fine-tuning complete! Model ID:
ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlu4dkL


### First Prompt Test that kinda worked (ran several before this) 

The problem with this one is that it did not show all values of actuals and forecast and had a very lengthy explanation for the analysis.

In [89]:
import openai
from openai import OpenAI
import numpy as np
import pandas as pd

# Step 1: Recreate the df from 2020 to 2024
np.random.seed(42)
weeks = pd.date_range("2020-01-06", "2024-12-30", freq='W-MON')
df = pd.DataFrame({'week_start': weeks})
df['year'] = df['week_start'].dt.year
df['week'] = df['week_start'].dt.isocalendar().week
df['seasonality'] = df['week'].apply(lambda x: np.sin(2 * np.pi * x / 52))
df['trend'] = np.linspace(1000, 3000, len(df))
df['marketing_event'] = df['week_start'].dt.month.isin([1, 8]).astype(int)
df['volume'] = df['trend'] + 300 * df['seasonality'] + df['marketing_event'] * 600 + np.random.normal(0, 100, len(df))

# Step 2: Format df as CSV-style string
df_text = "week_start,volume\n" + "\n".join([
    f"{row['week_start'].date()},{int(row['volume'])}" for _, row in df.iterrows()
])

# Step 3: Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 4: Construct the prompt
user_prompt = (
    "You are a WFM forecasting expert.\n\n"
    "Here is weekly contact center volume data from January 2020 through December 2024 in CSV format:\n\n"
    f"{df_text}\n\n"
    "Please forecast the weekly volume for each week of 2025.\n"
    "Then compare the forecast to realistic actuals (which you generate), display both side-by-side in a table, "
    "and calculate MAPE (Mean Absolute Percentage Error) and MAE (Mean Absolute Error) across the year. "
    "Conclude with a brief explanation of any trend, seasonal pattern, or outliers observed in your forecast."
)

# Step 5: Call your fine-tuned model
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlaxlD6",
    messages=[
        {"role": "system", "content": "You are a WFM forecasting expert."},
        {"role": "user", "content": user_prompt}
    ],
    max_tokens=4096
)

# Step 6: Output result
print("🔮 Forecasted Response:\n")
print(response.choices[0].message.content)


🔮 Forecasted Response:

Certainly, I can help you. I will first conduct the forecast, then compare the results to generate actuals, then calculate MAPE and MAE across the year. Afterwards, I will offer an explanation for any patterns observed in the forecast.

Please note: Your data will be reverse-engineered since I can’t produce the exact monthly volumes, but can produce some realistic ones.

Here is the forecast for 2025:

week_start,volume_forecast
2025-01-06,3200
2025-01-13,3400
2025-01-20,3300
2025-01-27,3300
... (remaining weeks of 2025)

I will now utilize the forecast and generate realistic actuals. I will produce the year’s average weekly volume in order to simulate a realistic pattern.

Actuals: I will use a mean of 2900 across the whole year, with some fluctuation due to seasonality. For instance, applying a 20-30% increase for January due to post-holiday volume might lead to around 3400; I will then modify by a factor between 0.85 and 1.15 to simulate historical fluctuatio

### Second Test

This test performed poorly as you can see now data was outputed in regards to forecast and actuals.

In [146]:
# Step 1: Recreate the df from 2020 to 2024
np.random.seed(42)
weeks = pd.date_range("2020-01-06", "2024-12-30", freq='W-MON')
df = pd.DataFrame({'week_start': weeks})
df['year'] = df['week_start'].dt.year
df['week'] = df['week_start'].dt.isocalendar().week
df['seasonality'] = df['week'].apply(lambda x: np.sin(2 * np.pi * x / 52))
df['trend'] = np.linspace(1000, 3000, len(df))
df['marketing_event'] = df['week_start'].dt.month.isin([1, 8]).astype(int)
df['volume'] = df['trend'] + 300 * df['seasonality'] + df['marketing_event'] * 600 + np.random.normal(0, 100, len(df))

# Step 2: Format df as CSV-style string
df_text = "week_start,volume\n" + "\n".join([
    f"{row['week_start'].date()},{int(row['volume'])}" for _, row in df.iterrows()
])

# Step 3: Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 4: Construct the prompt
user_prompt = (
    "You are a WFM forecasting expert.\n\n"
    "Here is weekly contact center volume data from January 2020 through December 2024 in CSV format:\n\n"
    f"{df_text}\n\n"
    "Please forecast the weekly volume for each week of 2025.\n"
    "Then compare the forecast to realistic actuals (which you generate), display both side-by-side in a table, "
    "and calculate MAPE (Mean Absolute Percentage Error) and MAE (Mean Absolute Error) across the year. "
    "Conclude with a brief explanation of any trend, seasonal pattern, or outliers observed in your forecast."
)

# Step 5: Call your fine-tuned model
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlaxlD6",
    messages=[
        {"role": "system", "content": "You are a WFM forecasting expert."},
        {"role": "user", "content": user_prompt}
    ],
    max_tokens=4096
)

# Step 6: Output result
print("Forecasted Response:\n")
print(response.choices[0].message.content)


Forecasted Response:

Got it. I'll get started on the analysis and get back to you shortly.


### Third Prompt Test

This one worked a lot better and showed the entire output I wanted. After this I adusted my fine tuned model with the hopes that it always outputs like this.

In [154]:

# Step 1: Recreate the df from 2020 to 2024
np.random.seed(42)
weeks = pd.date_range("2020-01-06", "2024-12-30", freq='W-MON')
df = pd.DataFrame({'week_start': weeks})
df['year'] = df['week_start'].dt.year
df['week'] = df['week_start'].dt.isocalendar().week
df['seasonality'] = df['week'].apply(lambda x: np.sin(2 * np.pi * x / 52))
df['trend'] = np.linspace(1000, 3000, len(df))
df['marketing_event'] = df['week_start'].dt.month.isin([1, 8]).astype(int)
df['volume'] = df['trend'] + 300 * df['seasonality'] + df['marketing_event'] * 600 + np.random.normal(0, 100, len(df))

# Step 2: Format df as CSV-style string
df_text = "week_start,volume\n" + "\n".join([
    f"{row['week_start'].date()},{int(row['volume'])}" for _, row in df.iterrows()
])

# Step 3: Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 4: Construct the prompt
user_prompt = (
    "You are a WFM forecasting expert.\n\n"
    "Here is weekly contact center volume data from January 2020 through December 2024 in CSV format:\n\n"
    f"{df_text}\n\n"
    "Please forecast the weekly volume for each week of 2025.\n"
    "Then compare the forecast to realistic actuals (which you generate), display both side-by-side in a table, "
    "and calculate MAPE (Mean Absolute Percentage Error) and MAE (Mean Absolute Error) across the year. "
    "Conclude with a brief explanation of any trend, seasonal pattern, or outliers observed in your forecast."
)

# Step 5: Call your fine-tuned model
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlaxlD6",
    messages=[
        {"role": "system", "content": "You are a WFM forecasting expert."},
        {"role": "user", "content": user_prompt}
    ],
    max_tokens=4096
)

# Step 6: Output result
print("🔮 Forecasted Response:\n")
print(response.choices[0].message.content)


🔮 Forecasted Response:

I've forecasted the weekly volumes for 2025. Given the unique nature of the contact center data, I'll generate realistic actuals as well.

*2025 Forecast (Volumes in Forecasts column, Actuals in Realistic Actuals column, MAPE and MAE calculated at the bottom)*

|Week Start|Forecasts|Realistic Actuals|
|----------|----------|------------------|
|2025-01-06|3250      |3300              |
|2025-01-13|3465      |3500              |
|2025-01-20|3520      |3450              |
|2025-01-27|3588      |3600              |
|2025-02-03|2954      |2900              |
|2025-02-10|2820      |2800              |
|2025-02-17|2890      |2900              |
|2025-02-24|2785      |2750              |
|2025-03-03|2873      |2900              |
|2025-03-10|2850      |2800              |
|2025-03-17|2995      |3000              |
|2025-03-24|2960      |2950              |
|2025-03-31|3100      |3100              |
|2025-04-07|2970      |2975              |
|2025-04-14|3110      |3100 

### Fourth Prompt Example 

Prior to running this block of code, I updated the examples of fune tuning my model with the goal of always having all actuals and forecast show. It appears to have worked!

In [114]:

# Step 1: Recreate the df from 2020 to 2024
np.random.seed(42)
weeks = pd.date_range("2020-01-06", "2024-12-30", freq='W-MON')
df = pd.DataFrame({'week_start': weeks})
df['year'] = df['week_start'].dt.year
df['week'] = df['week_start'].dt.isocalendar().week
df['seasonality'] = df['week'].apply(lambda x: np.sin(2 * np.pi * x / 52))
df['trend'] = np.linspace(1000, 3000, len(df))
df['marketing_event'] = df['week_start'].dt.month.isin([1, 8]).astype(int)
df['volume'] = df['trend'] + 300 * df['seasonality'] + df['marketing_event'] * 600 + np.random.normal(0, 100, len(df))

# Step 2: Format df as CSV-style string
df_text = "week_start,volume\n" + "\n".join([
    f"{row['week_start'].date()},{int(row['volume'])}" for _, row in df.iterrows()
])

# Step 3: Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 4: Construct the prompt
user_prompt = (
    "You are a WFM forecasting expert.\n\n"
    "Here is weekly contact center volume data from January 2020 through December 2024 in CSV format:\n\n"
    f"{df_text}\n\n"
    "Please forecast the weekly volume for each week of 2025.\n"
    "Then compare the forecast to realistic actuals (which you generate), display both side-by-side in a table, "
    "and calculate MAPE (Mean Absolute Percentage Error) and MAE (Mean Absolute Error) across the year. "
    "Conclude with a brief explanation of any trend, seasonal pattern, or outliers observed in your forecast."
)

# Step 5: Call your fine-tuned model
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlu4dkL",
    messages=[
        {"role": "system", "content": "You are a WFM forecasting expert."},
        {"role": "user", "content": user_prompt}
    ],
    max_tokens=4096
)

# Step 6: Output result
print("🔮 Forecasted Response:\n")
print(response.choices[0].message.content)


🔮 Forecasted Response:

First, I will define the historical period and use forecasting techniques to predict the volume for each week of 2025. Afterward, I generate realistic, but random, actuals for 2025, and then provide a comparison between the forecasted and actual volumes. I also calculate MAPE and MAE to evaluate the quality of the forecast. Finally, I include a brief analysis of trends, seasonality, or any notable outliers.

Starting with the historical data, I will use an appropriate forecasting model to generate weekly forecasts for 2025. Afterward, I generate actual volume values for each week using the generated forecast as a seed for realistic fluctuations around the forecasted level.

Week | Forecast | Actual
====|====|====
2025-01-06 | 3100 | 3209
2025-01-13 | 3050 | 3241
2025-01-20 | 3000 | 3098
2025-01-27 | 3150 | 3307
2025-02-03 | 3250 | 3132
2025-02-10 | 3300 | 3288
2025-02-17 | 3400 | 3332
2025-02-24 | 3200 | 3101
2025-03-02 | 3150 | 3097
2025-03-09 | 3100 | 3010
202

### Prompt Example 5 - Adjusted the prompt to change format of output

Finally, in this example, the model has been further fine-tune and the prompt changed to have a format that I prefer. So far it seems to be working bot not all the time. 

In [138]:

# Step 1: Recreate the df from 2020 to 2024
np.random.seed(42)
weeks = pd.date_range("2020-01-06", "2024-12-30", freq='W-MON')
df = pd.DataFrame({'week_start': weeks})
df['year'] = df['week_start'].dt.year
df['week'] = df['week_start'].dt.isocalendar().week
df['seasonality'] = df['week'].apply(lambda x: np.sin(2 * np.pi * x / 52))
df['trend'] = np.linspace(1000, 3000, len(df))
df['marketing_event'] = df['week_start'].dt.month.isin([1, 8]).astype(int)
df['volume'] = df['trend'] + 300 * df['seasonality'] + df['marketing_event'] * 600 + np.random.normal(0, 100, len(df))

# Step 2: Format df as CSV-style string
df_text = "week_start,volume\n" + "\n".join([
    f"{row['week_start'].date()},{int(row['volume'])}" for _, row in df.iterrows()
])

# Step 3: Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Step 4: Construct the prompt
user_prompt = (
    "You are a WFM forecasting expert.\n\n"
    "Here is weekly contact center volume data from January 2020 through December 2024 in CSV format:\n\n"
    f"{df_text}\n\n"
    "Please forecast weekly volume for 2025. Generate realistic actuals. Output your result in the following format and always show the entire output:\n"
    "Week Date | Forecast | Actuals\n\n"
    "Then calculate:\nMAPE:\nMAE:\nWeekly Variance Percentage:\nTotal Average Variance Percentage:\n\n"
    "Finish with a paragraph summarizing the trend, seasonality, cycles, and outliers you observe using percentages, then add how this impacts staffing levels\n\n."
)

# Step 5: Call your fine-tuned model
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:personal:wfm-predictor:BVlu4dkL",
    messages=[
        {"role": "system", "content": "You are a WFM forecasting expert."},
        {"role": "user", "content": user_prompt}
    ],
    max_tokens=4096
)

# Step 6: Output result
print("🔮 Forecasted Response:\n")
print(response.choices[0].message.content)


🔮 Forecasted Response:

Week Date | Forecast | Actuals
2025-01-06 | 3212 | 3550
2025-01-13 | 3289 | 3301
2025-01-20 | 2933 | 2800
2025-01-27 | 3071 | 2970
2025-02-03 | 3235 | 2598
2025-02-10 | 3323 | 3482
2025-02-17 | 3112 | 3032
2025-02-24 | 3149 | 3039
2025-03-02 | 3243 | 3110
2025-03-09 | 3044 | 3170
2025-03-16 | 2944 | 2972
2025-03-23 | 2981 | 3025
2025-03-30 | 3098 | 3102
2025-04-06 | 3284 | 3490
2025-04-13 | 3201 | 3150
2025-04-20 | 2875 | 2790
2025-04-27 | 3025 | 2925
2025-05-04 | 3180 | 3310
2025-05-11 | 3260 | 3200
2025-05-18 | 3030 | 3105
2025-05-25 | 3050 | 2900
2025-06-01 | 3165 | 3201
2025-06-08 | 3340 | 3489
2025-06-15 | 3000 | 2801
2025-06-22 | 3067 | 3126
2025-06-29 | 3183 | 3292
2025-07-06 | 3300 | 3511
2025-07-13 | 3435 | 3298
2025-07-20 | 3244 | 3200
2025-07-27 | 3292 | 3125
2025-08-03 | 3480 | 3600
2025-08-10 | 3568 | 3455
2025-08-17 | 3301 | 3180
2025-08-24 | 3365 | 3210
2025-08-31 | 3390 | 3500
2025-09-07 | 3540 | 3650
2025-09-14 | 3362 | 3170
2025-09-21 | 3293 | 

### Evaluation

Fine-tuning my model involved multiple iterative steps and refinements. Initially, when I created prompt examples, the model's output was limited to generic text responses without providing any forecasts or actuals. In fact, it often responded with placeholder language indicating that it would begin the task shortly. After significantly revising the examples in my fine-tuned dataset, the model began generating forecasts and actuals; however, the results were inconsistent. In some instances, it only produced data for a few weeks. To address this, I further refined the training examples to include explicit formatting for forecasts and actuals. This led to improved performance, although the output still varied considerably depending on the prompt used. Ultimately, I enhanced the prompt during inference to specify the desired output format clearly. This combination of fine-tuning and prompt engineering yielded the most reliable results, as demonstrated in my final example. Overall, this project provided a comprehensive learning experience in the end-to-end process of fine-tuning a language model and optimizing prompts to achieve targeted outputs. I was particularly impressed by the model’s ability to analyze data for trends, seasonality, outliers, and cyclical patterns.